In [2]:
from utils.block import BlockRes
from tensorflow.keras import layers
from utils.common import compose
import tensorflow as tf

In [19]:
import tensorflow.keras.backend as K

def Relu(x):
    return K.relu(x, max_value=6.0, alpha=0, threshold=0)

In [20]:
def CBN(
    filters, kernel_size=3,
    activation='relu', strides=1,
    padding="valid"):
    
    conv=layers.Conv2D(
        filters, kernel_size = kernel_size,
        strides = strides, padding = padding)
    bn=layers.BatchNormalization(epsilon = 1.001e-05)
    layers_list = [conv, bn]
    if activation:
        act=layers.Activation(activation = activation)
        layers_list.append(act)
    return compose(*layers_list)

In [21]:
def BlockMobile(
    kernel_size, filters, 
    padding="valid", 
    strides=1, activation=Relu
    ):

    layers_list=[
        layers.DepthwiseConv2D(
            kernel_size, strides=strides, 
            padding = padding, use_bias=False),
        layers.BatchNormalization( ),
        layers.Activation(activation=Relu),
        # 인셉션 넷과 동일하게 마지막에 1x1 필터를 적용
        layers.Conv2D(filters, 1, padding="same", use_bias=False), 
        layers.BatchNormalization( ),
        layers.Activation(activation=Relu),
    ]
    block=tf.keras.Sequential(layers_list)
    return block

In [38]:
input_shape = (224,224,3)

inp = layers.Input(shape = input_shape)

In [39]:
x=tf.keras.Sequential(
[
    layers.Conv2D(32, 3, strides=2, padding="same", use_bias= False),
    layers.BatchNormalization( ),
    layers.Activation(tf.keras.activations.relu),# tf.nn.relu6
])(inp)

In [40]:
filters = [64, 128, 256, 512, 1024]
iters = [1,2,2,6,2]
flat_fun = tf.keras.layers.GlobalAveragePooling2D
for i, (filter, iter) in enumerate(zip(filters, iters)):
    for j in range(iter):
        padding = "valid" if i != 0 and j == 0 else "same"
        strides = 2 if i != 0 and j == 0 else 1
        x=BlockMobile(
            3, filter, padding=padding, strides = strides)(x)
    if len(iters) != i+1:
        x=layers.ZeroPadding2D(padding=((0,1),(0,1)))(x)
x = flat_fun()(x)
out = layers.Dense( 10, activation = "softmax")(x)
tf.keras.Model(inp, out).summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential_76 (Sequential)  (None, 112, 112, 32)      992       
                                                                 
 sequential_77 (Sequential)  (None, 112, 112, 64)      2720      
                                                                 
 zero_padding2d_23 (ZeroPadd  (None, 113, 113, 64)     0         
 ing2D)                                                          
                                                                 
 sequential_78 (Sequential)  (None, 56, 56, 128)       9536      
                                                                 
 sequential_79 (Sequential)  (None, 56, 56, 128)       18560     
                                                           